In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Only keeping the necessary columns.

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Major Project/LSTM-keras/DataSet.csv')


Data preview

In [11]:
data.head()

,text,sentiment
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,negative
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,negative
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,negative
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,negative
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,negative


Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [12]:
print(data[ data['sentiment'] == 'positive'].size)
print(data[ data['sentiment'] == 'negative'].size)
print(data[ data['sentiment'] == 'neutral'].size)


6508
6348
5298


In [14]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  100,   46,  308,   18,  309,  297,  434, 1259,  206,
        1726,   29,  297,  831,  968],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    4,   12,   74,  100,
           8, 1472, 1727,   24,  969]], dtype=int32)

Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [26]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 38, 128)           256000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 38, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 196)               254800    
                                                                 
 dense_1 (Dense)             (None, 3)                 591       
                                                                 
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [27]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7261, 38) (7261, 3)
(1816, 38) (1816, 3)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [28]:
model.fit(x=X_train, y=Y_train, epochs = 10, batch_size=128, verbose = 1)

Epoch 1/10
57/57 [==============================] - 32s 504ms/step - loss: 0.8571 - accuracy: 0.5711
Epoch 2/10
57/57 [==============================] - 29s 508ms/step - loss: 0.6080 - accuracy: 0.7184
Epoch 3/10
57/57 [==============================] - 29s 507ms/step - loss: 0.4595 - accuracy: 0.8138
Epoch 4/10
57/57 [==============================] - 42s 742ms/step - loss: 0.3591 - accuracy: 0.8533
Epoch 5/10
57/57 [==============================] - 29s 507ms/step - loss: 0.3201 - accuracy: 0.8714
Epoch 6/10
57/57 [==============================] - 29s 504ms/step - loss: 0.2875 - accuracy: 0.8853
Epoch 7/10
57/57 [==============================] - 30s 527ms/step - loss: 0.2644 - accuracy: 0.8944
Epoch 8/10
57/57 [==============================] - 29s 506ms/step - loss: 0.2413 - accuracy: 0.9041
Epoch 9/10
57/57 [==============================] - 29s 511ms/step - loss: 0.2218 - accuracy: 0.9131
Epoch 10/10
57/57 [==============================] - 29s 507ms/step - loss: 0.2048 - accura

Extracting a validation set, and measuring score and accuracy.

In [30]:
Y_pred = model.predict(X_test,batch_size = 128)

In [31]:
classes_Y=np.argmax(Y_pred,axis=1)

In [33]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':classes_Y})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[532  23 103]
 [ 33 470  32]
 [124  27 472]]
              precision    recall  f1-score   support

           0       0.77      0.81      0.79       658
           1       0.90      0.88      0.89       535
           2       0.78      0.76      0.77       623

    accuracy                           0.81      1816
   macro avg       0.82      0.81      0.82      1816
weighted avg       0.81      0.81      0.81      1816

